In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

Using TensorFlow backend.


There are multiple ways of loading data depending on where you load it from. Just run the relevant code block below (or add your own)

Note: I am not sure about  order of psi and phi. 
index => 0,1,2,3,...
pdb => unique name of protein
pdb_aa => amino acid sequence over 21 chars
q8 => sequence over eight chars
dist_matrix =>  ouput 1

FOR PART II
outputs are psi and phi

In [4]:
def load_pickle(datafile):
    # load train data for a specific fold
    with open(datafile, 'rb') as f:
        data =  pickle.load(f)

        
    return data

In [5]:
indices, pdbs, length_aas, pdb_aas, q8s, dcalphas, psis, phis, msas = load_pickle("../data/train_fold_3.pkl")

In [20]:
length_aas[2]

108

In [8]:
dcalphas[0].shape

(162, 162)

In [56]:
#get the max length of some input sequence
max_length = max(length_aas)

Data Preprocessing

In [10]:
from keras.preprocessing import sequence

In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
aa_list = list('ACEDGFIHKMLNQPSRTWVYX') + ['NoSeq'] #CHECK THIS
q8_list      = list('LBEGIHST') + ['NoSeq']

In [13]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence

In [15]:
vocab_size = len(aa_list)

In [16]:
text = " ".join(list(pdb_aas[2]))

In [17]:
result = one_hot(text, round(vocab_size))

In [30]:
result = np.array([result])

In [31]:
result.shape

(1, 108)

In [32]:
max_length = 120

In [35]:
input_train = sequence.pad_sequences(result, maxlen=max_length, padding = "post")

In [36]:
input_train

array([[ 9,  3,  3, 16, 12, 11, 12, 16, 16,  5, 16, 11,  9, 16, 11, 11,
        13,  3, 11,  3, 13, 20, 15,  3, 16, 16, 15, 16, 16,  6, 15, 12,
        12, 11, 11,  3,  9, 13, 11, 15, 16, 12, 18, 16, 16, 12, 12,  3,
         3, 16,  3, 11,  3,  5,  9,  3, 16,  6, 11,  5,  9,  6,  3,  9,
         3, 20,  9, 20,  3, 13, 13, 11, 12,  3,  3, 11, 15,  3, 13, 15,
         5,  5, 13, 16, 16,  6, 15,  9, 20, 13,  3,  3, 18,  6,  9,  5,
         3, 19,  5,  9, 16,  5, 15,  5,  6,  3, 12,  9,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0]], dtype=int32)

In [132]:
pdb_aas[2]

'MAAGELEGGKPLSGLLNALAQDTFHGYPGITEELLRSQLYPEVPPEEFRPFLAKMRGILKSIASADMDFNQLEAFLTAQTKKQGGITSDQAAVISKFWKSHKTKIRES'

In [17]:
def produce_test_output(testfile, save_path):
    test_input = pd.read_csv(testfile, header=None)
    protein_names = np.array(test_input.iloc[:,1])
    protein_len = np.array(test_input.iloc[:,2])

    # concatenate all output to one-dimensional
    all_data = []
    all_names = []
    for i, pname in enumerate(protein_names):
        dist_flat = dist[i].ravel()
        array = np.concatenate([dist_flat, psi[i], phi[i]])
        all_data.append(array)

        length = protein_len[i]
        dist_names = ["{}_d_{}_{}".format(pname, i + 1, j + 1) for i in range(length) for
                j in range(length)]

        psi_names = ["{}_psi_{}".format(pname, i + 1) for i in range(length)]
        phi_names = ["{}_phi_{}".format(pname, i + 1) for i in range(length)]
        row_names = np.array(dist_names + psi_names + phi_names)
        all_names.append(row_names)

    all_data = np.concatenate(all_data)
    all_names = np.concatenate(all_names)
    output = {"Id": all_names, "Predicted": all_data}
    output = pd.DataFrame(output)
    output.to_csv(save_path, index=False)